In [1]:
!pip install ultralytics -q
!pip install roboflow -q


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ultralytics.nn.modules import (
    AIFI,
    C1,
    C2,
    C2PSA,
    C3,
    C3TR,
    ELAN1,
    OBB,
    PSA,
    SPP,
    SPPELAN,
    SPPF,
    A2C2f,
    AConv,
    ADown,
    Bottleneck,
    BottleneckCSP,
    C2f,
    C2fAttn,
    C2fCIB,
    C2fPSA,
    C3Ghost,
    C3k2,
    C3x,
    CBFuse,
    CBLinear,
    Classify,
    Concat,
    Conv,
    Conv2,
    ConvTranspose,
    Detect,
    DWConv,
    DWConvTranspose2d,
    Focus,
    GhostBottleneck,
    GhostConv,
    HGBlock,
    HGStem,
    ImagePoolingAttn,
    Index,
    LRPCHead,
    Pose,
    RepC3,
    RepConv,
    RepNCSPELAN4,
    RepVGGDW,
    ResNetLayer,
    RTDETRDecoder,
    SCDown,
    Segment,
    TorchVision,
    WorldDetect,
    YOLOEDetect,
    YOLOESegment,
    v10Detect,
)

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('../../myconfig.config')
print(config['roboflow']['roboflow_api_key'])


KeyError: 'roboflow'

In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key='u8Ur5ywKLswqwt0uMTXl')
project = rf.workspace("poacher-ifyuf").project("poacher")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.166, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to poacher-2 in yolov8:: 100%|██████████| 1396/1396 [00:00<00:00, 2020.27it/s]


In [ ]:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab_Notebooks/YOLO_architecture/utillity')

In [4]:
from ultralytics import nn
from ultralytics.nn import modules, tasks
import ultralytics.nn.autobackend as autobackend
from model_classes import Upsample, Conv, C2f, SPPF, DFL, Head, Bottleneck, Concat
from ultralytics.nn.tasks import parse_model, yaml_model_load
from copy import deepcopy


def patch_ultralytics_layers(custom_classes):
    submodules = [
        nn,                          # ultralytics.nn
        modules,                     # ultralytics.nn.modules
        tasks,                       # ultralytics.nn.tasks
        autobackend,                 # ultralytics.nn.autobackend
    ]

    for cls in custom_classes:
        for mod in submodules:
            mod.__dict__[cls.__name__] = cls

def build_model(model_yaml, version='n', ch=3, verbose=False, custom_classes=[]):

  patch_ultralytics_layers(custom_classes)

  mod_ya = model_yaml if isinstance(model_yaml, dict) else yaml_model_load(model_yaml)
  mod_ya['scale'] = version.lower()
  model, save  = parse_model(deepcopy(mod_ya), ch=ch, verbose=verbose)

  return model,save


In [5]:
from ultralytics import YOLO
m_class = [Upsample, Conv, C2f, SPPF, DFL, Head, Bottleneck, Concat]
model_yaml = "my_yolov8l.yaml"

model,save = build_model(model_yaml, custom_classes=m_class, version='l', verbose=False)

model = YOLO(model_yaml, verbose=True)
print(f"{sum(p.numel() for p in model.parameters()) / 1e6:.2f} million parameters")


                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  model_classes.Conv                           [3, 64, 3, 2]                 
  1                  -1  1     73984  model_classes.Conv                           [64, 128, 3, 2]               
  2                  -1  3    279808  model_classes.C2f                            [128, 128, 3, True]           
  3                  -1  1    295424  model_classes.Conv                           [128, 256, 3, 2]              
  4                  -1  6   2101248  model_classes.C2f                            [256, 256, 6, True]           
  5                  -1  1   1180672  model_classes.Conv                           [256, 512, 3, 2]              
  6                  -1  6   8396800  model_classes.C2f                            [512, 512, 6, True]           
  7                  -1  1   2360320  model_classes.Conv                           [512

In [6]:
for m in model.modules():
  if isinstance(m, (Conv, Conv2, DWConv)) and hasattr(m, "bn"):
    if isinstance(m, Conv2):
      pass
    print(m.conv)
    break


Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)


In [9]:
results = model.train(data=r"D:\WORK\project 1\Architecture\Test_Dataset\data.yaml", epochs=1, imgsz=64)

New https://pypi.org/project/ultralytics/8.3.189 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.166  Python-3.11.1 torch-2.4.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\WORK\project 1\Architecture\Test_Dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=my_yolov8l.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, n

train: Scanning D:\WORK\project 1\Architecture\Test_Dataset\train\labels... 604 images, 9 backgrounds, 0 corrupt: 100%|██████████| 604/604 [00:02<00:00, 242.67it/s]

train: New cache created: D:\WORK\project 1\Architecture\Test_Dataset\train\labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.20.3 ms, read: 5.92.3 MB/s, size: 56.0 KB)


val: Scanning D:\WORK\project 1\Architecture\Test_Dataset\valid\labels... 58 images, 1 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<00:00, 254.39it/s]

...

Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 0 dataloader workers
Logging results to runs\detect\train4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      6.446      5.152      4.015         48         64: 100%|██████████| 38/38 [00:37<00:00,  1.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]


                   all         58        122          0          0          0          0

1 epochs completed in 0.012 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 87.6MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 87.6MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.166  Python-3.11.1 torch-2.4.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
my_YOLOv8l summary: 209 layers, 43,625,490 parameters, 0 gradients, 164.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.37it/s]


                   all         58        122          0          0          0          0
                animal          2          2          0          0          0          0
                  bull          1          1          0          0          0          0
                 camel          2          4          0          0          0          0
                  deer          3          3          0          0          0          0
                   dog          2          3          0          0          0          0
              elephant          8          8          0          0          0          0
                   fox          1          1          0          0          0          0
                  lion          2          2          0          0          0          0
               poacher         18         29          0          0          0          0
                ranger         18         32          0          0          0          0
                 rhin